In [1]:
!pip install accelerate transformers langchain torch bitsandbytes einops huggingface_hub

In [2]:
!pip install faiss-cpu

In [3]:
import getpass
from huggingface_hub import login
hf_token = getpass.getpass("Enter your Hugging Face token: ")
login(token=hf_token)

Enter your Hugging Face token:  ········


In [46]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM
# Set local_files_only=False to allow downloading the model
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B", local_files_only=False)
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B", local_files_only=False)

In [5]:
!pip install langchain-community
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

<ipython-input-5-25181f2886c5>:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")


In [6]:
!pip install unstructured
from langchain.document_loaders import UnstructuredURLLoader
URLs=[
    'https://blog.gopenai.com/paper-review-llama-2-open-foundation-and-fine-tuned-chat-models-23e539522acb',
    'https://www.datacamp.com/tutorial/llama3-fine-tuning-locally']
loader=UnstructuredURLLoader(URLs)
docs=loader.load()

In [10]:
len(docs),type(docs[0])

(2, langchain_core.documents.base.Document)

In [11]:
!pip install langchain-text-splitters
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    separators=['\n'],
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)
docs=text_splitter.split_documents(docs)

In [18]:
len(docs),type(docs)

(30, list)

In [32]:
!pip install chromadb
from langchain.vectorstores import Chroma
persist_directory = 'db'
vectordb = Chroma.from_documents(documents=docs,
                                 embedding=embeddings,
                                 persist_directory=persist_directory)

In [35]:
vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=embeddings)
retriever = vectordb.as_retriever()

<ipython-input-35-e67af6cd9793>:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(persist_directory=persist_directory,


In [37]:
retriever.get_relevant_documents("How to fine tune llama locally",k=1)

[Document(metadata={'source': 'https://blog.gopenai.com/paper-review-llama-2-open-foundation-and-fine-tuned-chat-models-23e539522acb'}, page_content='Fine-tuning\n\nSupervised Fine-Tuning (SFT)\n\nThe authors initiated the Supervised Fine-Tuning (SFT) phase using publicly available instruction tuning data like in Llama. However, they observed that many third-party SFT data sources lacked diversity and quality, particularly for aligning Large Language Models (LLMs) towards dialogue-style instructions. Therefore, they prioritized collecting several thousand high-quality SFT data examples, and found that using fewer but better-quality examples led to notable performance improvements.')]

In [40]:
!pip install torch
import torch
from transformers import pipeline
pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=4,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )

Device set to use cuda:0


In [41]:
from langchain import HuggingFacePipeline
llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0.1})

<ipython-input-41-7793df4e31ba>:2: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0.1})


In [43]:
from langchain.chains import RetrievalQA
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vectordb.as_retriever())

In [44]:
qa.run("how we can fine tune llama locally")

<ipython-input-44-4b32f503a51c>:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  qa.run("how we can fine tune llama locally")
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


"Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nFine-tuning\n\nSupervised Fine-Tuning (SFT)\n\nThe authors initiated the Supervised Fine-Tuning (SFT) phase using publicly available instruction tuning data like in Llama. However, they observed that many third-party SFT data sources lacked diversity and quality, particularly for aligning Large Language Models (LLMs) towards dialogue-style instructions. Therefore, they prioritized collecting several thousand high-quality SFT data examples, and found that using fewer but better-quality examples led to notable performance improvements.\n\nIterative Fine-Tuning\n\nTwo main algorithms were used for RLHF fine-tuning: Proximal Policy Optimization (PPO), standard in RLHF literature, and Rejection Sampling fine-tuning, where they selected the best output from sampled model responses for a gradient update. The differences between 